In [1]:
import os
os.chdir(os.path.expanduser("~/Dev/personal/is_it_ai"))
from utils.data_ops import get_whole_dataset
import gc
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import optuna
import gc

/home/mcygert/Dev/personal/is_it_ai/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_path = "data/"
full_training_data = get_whole_dataset(root_path)
X = full_training_data["text"]
y = full_training_data['label']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
LOWERCASE = False
VOCAB_SIZE = 30522

In [15]:
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
dataset = Dataset.from_dict({'text': x_test, 'label': y_test})
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]['text']
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(x_test):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = [] 
for text in tqdm(x_train):
    tokenized_texts_train.append(tokenizer.tokenize(text))

100%|██████████| 36996/36996 [00:26<00:00, 1380.79it/s]


In [ ]:
def dummy(text):
    return text
vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, analyzer = 'word',
    tokenizer = dummy,
    preprocessor = dummy,
    token_pattern = None, strip_accents='unicode')

vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()

{'Ġ [UNK] [UNK]': 12736, '[UNK] [UNK] Ġ': 3241, '[UNK] Ġ [UNK]': 5194, '[UNK] [UNK] [UNK]': 2706, 'Ġ [UNK] a': 12873, '[UNK] a [UNK]': 3383, 'a [UNK] [UNK]': 5666, '[UNK] [UNK] e': 3121, '[UNK] e [UNK]': 4511, 'e [UNK] Ġ': 10561, '[UNK] [UNK] a': 2851, 'a [UNK] Ġ': 5950, 'Ġ [UNK] e': 13035, 'e [UNK] [UNK]': 10244, '[UNK] Ġ a': 5312, 'Ġ a d': 13343, 'a d [UNK]': 6390, 'd [UNK] [UNK]': 8938, '[UNK] e d': 4715, 'e d Ġ': 11116, 'd Ġ [UNK]': 9684, 'e [UNK] e': 10490, '[UNK] e Ġ': 4767, 'e Ġ [UNK]': 11305, '[UNK] Ġ b': 5372, 'Ġ b e': 13553, 'b e Ġ': 7485, 'Ġ [UNK] d': 12994, '[UNK] d [UNK]': 4180, 'd [UNK] e': 9119, '[UNK] e .': 4484, 'e . .': 10079, '. . .': 0, '. . Ġ': 133, '. Ġ [UNK]': 611, '[UNK] [UNK] d': 3052, '[UNK] d e': 4297, 'd e [UNK]': 9391, '[UNK] [UNK] c': 2981, '[UNK] c [UNK]': 3891, 'c [UNK] Ġ': 8068, 'e Ġ d': 11516, 'Ġ d [UNK]': 13743, 'e Ġ a': 11409, 'Ġ a [UNK]': 13205, 'a [UNK] e': 5885, '[UNK] Ġ d': 5444, '[UNK] [UNK] .': 2551, '[UNK] . [UNK]': 2072, '. [UNK] Ġ': 396, 'e 

0

In [ ]:
clf = MultinomialNB(alpha=0.1)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6={'n_iter': 2500,
    'verbose': -1,
    'objective': 'cross_entropy',
    'metric': 'auc',
    'learning_rate': 0.01, 
    'colsample_bytree': 0.78,
    'colsample_bynode': 0.8, 
    'lambda_l1': 4.562963348932286, 
    'lambda_l2': 2.97485, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898}

lgb=LGBMClassifier(**p6)
weights = [0.068,0.311,0.31]

ensemble = VotingClassifier(estimators=[('mnb',clf),
                                        ('sgd', sgd_model),
                                        ('lgb',lgb)
                                        ],
                            weights=weights, voting='soft', n_jobs=-1)
ensemble.fit(tf_train, y_train)
gc.collect()
final_preds = ensemble.predict_proba(tf_test)[:,1]
final_preds

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

array([9.94650616e-01, 2.16215581e-04, 5.31539318e-02, ...,
       3.26892170e-04, 9.96383171e-01, 6.96044454e-04])

In [ ]:
print(f"{roc_auc_score(y_test, final_preds)} -> Roc_auc_score")

0.9989008976002931 -> Roc_auc_score
